In [2]:
# Setup
import matplotlib as plt
import numpy as np
from dolo import * 


In [4]:
filename = 'Aiyagari.yaml'
pcat(filename)

1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78


In [5]:
model = yaml_import(filename)

    calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.


DuplicateKeyError: while constructing a mapping
  in "<unicode string>", line 2, column 1:
    name: Agent
    ^ (line: 2)
found duplicate key "name" with value "Aggregate" (original value: "Agent")
  in "<unicode string>", line 59, column 1:
    name: Aggregate
    ^ (line: 59)

To suppress this check see:
    http://yaml.readthedocs.io/en/latest/api.html#duplicate-keys

Duplicate keys will become an error in future releases, and are errors
by default when using the new API.
